In [ ]:
import pandas as pd

In [ ]:
fuel_res = pd.read_excel("raw_data/Canada/res_ca_e_1.xls").dropna(how="all").drop([3,5,6], axis=0)
fuel_res.drop(fuel_res.columns[0], axis=1, inplace=True)
fuel_res.columns = fuel_res.iloc[0]
#fuel_res.columns = pd.to_datetime(fuel_res.iloc[0])
# still need to convert years to datetime
fuel_res